**Алгоритм LambdaMART**

**Тип:** pairwise метод

**Данные:** нужно обязательно указывать qid -- попарные сравнения производятся только внутри одного запроса (пользователя). Ограничений к данным нет

**Общий принцип работы:** 

Ранжирование превращается в задачу попарной классификации или регрессии: просматриваем пары элементов за раз, определяем оптимальный порядок для этой пары элементов, а затем используем его для определения окончательного ранжирования.

LambdaMart объединяет LambdaRank и MART. В алгоритме MART используется gradient boosted decision trees для prediction. LambdaMart использует gradient boosted decision trees с функцией cost (от алгоритма LambdaRank), для решения задачи ранжирования.

**Если подробнее:**

MART 

MART – это бустинг, сделанный на регрессионных деревьях

* Нам нужно понять, как обучать новое дерево, если мы уж обучили n-е количество деревьев
* Фиксируем функцию ошибки, следующее дерево моделирует производную ошибки 
* Выбираем оптимальный шаг(минимизируем потери)

LAMBDAMART

* Добавляем в градиенты целевую метрику (в критерий ошибки (какое разбиение лучше) добавляется оценка качества ранжирования, за счет чего получаем ошибку, которая завязана на качестве ранжирования - насколько вариант с таким порядком лучше, чем вариант с другим порядком одной и той же пары)
* Функция ошибки нам известна 
* В каждой точке собираем все «действующие силы» 
* Мы хотим сдвинуться в сторону минимума ошибки; значит,
надо идти в сторону нуля производной
* Веса бустинга подбираются как шаг метода
Ньютона для каждого листа дерева




Логика: рассматриваем положение документов друг относительно друга. Берем пару документов в рамках одного запроса и на основании имеющихся данных определяем какой документ должен быть раньше. За счет этого собираются результаты в pairwise оценке. 

Общая идея: специфицирцем ошибку того, что элементы находятся не в том порядке, далее минимизируем эту ошибку (наименьшее количество случаев, когда документы находятся не в правильнос порядке).

Первоначальный RankNet предполагает штраф за то, что элементы располагаются не в правильном порядке, далее происходит минимизация функции, которая описывает этот штраф, с помощью стохастического градиентного спуска.


В LambdaRank в функцию добавляется ndcg (что ускоряет результат).

В LambdaMART для вычислений используются внутрирегресионные деревья, на каждом этапе на основе результатов регр.деревьев происходит сдвиг документа - выше или ниже (за счет попарных сравнений). 




МОДЕЛЬ

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import numpy as np

def dcg(scores):
    """
    compute the DCG value based on the given score
    :param scores: a score list of documents
    :return v: DCG value
    """
    v = 0
    for i in range(len(scores)):
        v += (np.power(2, scores[i]) - 1) / np.log2(i+2)  # i+2 is because i starts from 0
    return v

def single_dcg(scores, i, j):
    """
    compute the single dcg that i-th element located j-th position
    :param scores:
    :param i:
    :param j:
    :return:
    """
    return (np.power(2, scores[i]) - 1) / np.log2(j+2)


In [ ]:

def idcg(scores):
    """
    compute the IDCG value (best dcg value) based on the given score
    :param scores: a score list of documents
    :return:  IDCG value
    """
    best_scores = sorted(scores)[::-1]
    return dcg(best_scores)


def ndcg(scores):
    """
    compute the NDCG value based on the given score
    :param scores: a score list of documents
    :return:  NDCG value
    """
    return dcg(scores)/idcg(scores)


In [ ]:
def ndcg_k(scores, k):
    scores_k = scores[:k]
    dcg_k = dcg(scores_k)
    idcg_k = dcg(sorted(scores)[::-1][:k])
    if idcg_k == 0:
        return np.nan
    return dcg_k/idcg_k

def group_by(data, qid_index):
    """
    :param data: input_data
    :param qid_index: the column num where qid locates in input data
    :return: a dict group by qid
    """
    qid_doc_map = {}
    idx = 0
    for record in data:
        qid_doc_map.setdefault(record[qid_index], [])
        qid_doc_map[record[qid_index]].append(idx)
        idx += 1
    return qid_doc_map


In [ ]:
def get_pairs(scores):
    """
    :param scores: given score list of documents for a particular query
    :return: the documents pairs whose firth doc has a higher value than second one.
    """
    pairs = []
    for i in range(len(scores)):
        for j in range(len(scores)):
            if scores[i] > scores[j]:
                pairs.append((i, j))
    return pairs

In [ ]:
def compute_lambda(true_scores, temp_scores, order_pairs, qid):
    """
    :param true_scores: the score list of the documents for the qid query
    :param temp_scores: the predict score list of the these documents
    :param order_pairs: the partial oder pairs where first document has higher score than the second one
    :param qid: specific query id
    :return:
        lambdas: changed lambda value for these documents
        w: w value
        qid: query id
    """
    doc_num = len(true_scores)
    lambdas = np.zeros(doc_num)
    w = np.zeros(doc_num)
    IDCG = idcg(true_scores)
    single_dcgs = {}
    for i, j in order_pairs:
        if (i, i) not in single_dcgs:
            single_dcgs[(i, i)] = single_dcg(true_scores, i, i)
        if (j, j) not in single_dcgs:
            single_dcgs[(j, j)] = single_dcg(true_scores, j, j)
        single_dcgs[(i, j)] = single_dcg(true_scores, i, j)
        single_dcgs[(j, i)] = single_dcg(true_scores, j, i)

    for i, j in order_pairs:
        delta = abs(single_dcgs[(i,j)] + single_dcgs[(j,i)] - single_dcgs[(i,i)] -single_dcgs[(j,j)])/IDCG
        rho = 1 / (1 + np.exp(temp_scores[i] - temp_scores[j]))
        lambdas[i] += rho * delta
        lambdas[j] -= rho * delta

        rho_complement = 1.0 - rho
        w[i] += rho * rho_complement * delta
        w[i] -= rho * rho_complement * delta

    return lambdas, w, qid


In [ ]:
class LambdaMART:

    def __init__(self, training_data=None, number_of_trees=10, lr = 0.001):
        self.training_data = training_data
        self.number_of_trees = number_of_trees
        self.lr = lr
        self.trees = []

    def fit(self):
        """
        train the model to fit the train dataset
        """
        qid_doc_map = group_by(self.training_data, 1)
        query_idx = qid_doc_map.keys()
        # true_scores is a matrix, different rows represent different queries
        true_scores = [self.training_data[qid_doc_map[qid], 0] for qid in query_idx]

        order_paris = []
        for scores in true_scores:
            order_paris.append(get_pairs(scores))

        sample_num = len(self.training_data)
        predicted_scores = np.zeros(sample_num)
        for k in range(self.number_of_trees):
            print('Tree %d' % k)
            lambdas = np.zeros(sample_num)
            w = np.zeros(sample_num)

            temp_score = [predicted_scores[qid_doc_map[qid]] for qid in query_idx]
            zip_parameters = zip(true_scores, temp_score, order_paris, query_idx)
            for ts, temps, op, qi in zip_parameters:
                sub_lambda, sub_w, qid = compute_lambda(ts, temps, op, qi)
                lambdas[qid_doc_map[qid]] = sub_lambda
                w[qid_doc_map[qid]] = sub_w
            tree = DecisionTreeRegressor(max_depth=50)
            tree.fit(self.training_data[:, 2:], lambdas)
            self.trees.append(tree)
            pred = tree.predict(self.training_data[:, 2:])
            predicted_scores += self.lr * pred

            # print NDCG
            qid_doc_map = group_by(self.training_data, 1)
            ndcg_list = []
            for qid in qid_doc_map.keys():
                subset = qid_doc_map[qid]
                sub_pred_score = predicted_scores[subset]

                # calculate the predicted NDCG
                true_label = self.training_data[qid_doc_map[qid], 0]
                topk = len(true_label)
                pred_sort_index = np.argsort(sub_pred_score)[::-1]
                true_labell = true_label[pred_sort_index]
                ndcg_val = ndcg_k(true_labell, topk)
                ndcg_list.append(ndcg_val)
            print('Epoch:{}, Average NDCG : {}'.format(k, np.nanmean(ndcg_list)))

    def predict(self, data):
        """
        predict the score for each document in testset
        :param data: given testset
        :return:
        """
        qid_doc_map = group_by(data, 1)
        predicted_scores = np.zeros(len(data))
        for qid in qid_doc_map.keys():
            sub_result = np.zeros(len(qid_doc_map[qid]))
            for tree in self.trees:
                sub_result += self.lr * tree.predict(data[qid_doc_map[qid], 2:])
            predicted_scores[qid_doc_map[qid]] = sub_result
        return predicted_scores

    def validate(self, data, k):
        """
        validate the NDCG metric
        :param data: given th testset
        :param k: used to compute the NDCG@k
        :return:
        """
        #группируем наши данные по qid
        qid_doc_map = group_by(data, 1)
        ndcg_list = []
        predicted_scores = np.zeros(len(data))
        for qid in qid_doc_map.keys():
            sub_pred_result = np.zeros(len(qid_doc_map[qid]))
            for tree in self.trees:
                sub_pred_result += self.lr * tree.predict(data[qid_doc_map[qid], 2:])
            # предсказанная релевантность
            predicted_scores[qid_doc_map[qid]] = sub_pred_result
            # calculate the predicted NDCG
            # релевантность, кот была 
            true_label = data[qid_doc_map[qid], 0] 
            # то, как нужно упорядочить документы (ранжирование)
            pred_sort_index = np.argsort(sub_pred_result)[::-1] 
            # релевантность в предсказанном порядке
            true_labell = true_label[pred_sort_index] 
            # выводим параметры при qid = 10032
            if qid == 10032.0:
              print("Данная релевантность: ", true_label)
              print("Порядок того как бы мы упорядочили документы : ", pred_sort_index)
              print("Как расположилась релевантность в предсказанном порядке: ", true_labell)

            ndcg_val = ndcg_k(true_labell, k)
            ndcg_list.append(ndcg_val)
        print('Epoch:{}, Average NDCG : {}'.format(k, np.nanmean(ndcg_list)))



Загружаем данные

In [ ]:
import pandas as pd
df_train = pd.read_csv('trainData.csv')
df_test = pd.read_csv('testData.csv')

In [ ]:
df_train.head()

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
0,0,11909,0.048537,0,0.054362,0.000000,0.000000,0.000000,0,0.208262,0.089286,1.000000
1,0,11909,0.000000,0,0.000000,0.000000,0.000000,0.080000,0,0.000000,0.000000,0.000000
2,0,11909,0.014989,0,0.005346,1.000000,1.000000,1.000000,0,1.000000,1.000000,0.166667
3,1,11909,0.048180,0,0.016753,0.000000,1.000000,0.253333,0,0.040667,0.017857,0.000000
4,2,11909,0.254818,0,0.135242,0.615723,0.333333,0.253333,0,0.004727,0.017857,0.527778


In [ ]:
df_test.head()

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
0,0,10032,0.031802,0,0.007235,0.0,0.50,0.46875,0,1.000000,1.00,0.153846
1,0,10032,0.000000,0,0.004242,0.0,0.50,0.25000,0,0.013136,0.25,0.461538
2,0,10032,0.014134,0,0.000000,0.0,0.25,1.00000,0,0.054746,0.25,0.000000
3,2,10032,0.067138,0,0.004012,0.0,0.00,0.12500,0,0.000000,0.00,0.076923
4,0,10032,0.279152,0,0.015094,0.0,0.50,0.00000,0,0.002186,0.25,1.000000


Конвертируем csv в numpy, чтобы использовать данные в алгоритме

In [ ]:
numpy_train = df_train[:].values
numpy_test = df_test[:].values
numpy_train

array([[0.00000e+00, 1.19090e+04, 4.85370e-02, ..., 2.08262e-01,
        8.92860e-02, 1.00000e+00],
       [0.00000e+00, 1.19090e+04, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [0.00000e+00, 1.19090e+04, 1.49890e-02, ..., 1.00000e+00,
        1.00000e+00, 1.66667e-01],
       ...,
       [0.00000e+00, 1.82180e+04, 2.02020e-02, ..., 1.00000e+00,
        1.00000e+00, 2.50000e-01],
       [0.00000e+00, 1.82180e+04, 6.06100e-03, ..., 1.11722e-01,
        0.00000e+00, 1.25000e-01],
       [0.00000e+00, 1.82180e+04, 1.00000e+00, ..., 6.42800e-03,
        0.00000e+00, 0.00000e+00]])

Выбираем определенный запрос из тестовой выбоки

In [ ]:
print(df_test.loc[df_test.qid == 10032])

   relevance    qid        tf  idf  ...  outlink     slash  urlLength  childPage
0          0  10032  0.031802    0  ...        0  1.000000       1.00   0.153846
1          0  10032  0.000000    0  ...        0  0.013136       0.25   0.461538
2          0  10032  0.014134    0  ...        0  0.054746       0.25   0.000000
3          2  10032  0.067138    0  ...        0  0.000000       0.00   0.076923
4          0  10032  0.279152    0  ...        0  0.002186       0.25   1.000000
5          0  10032  0.134276    0  ...        0  0.023578       0.75   1.000000
6          1  10032  0.600707    0  ...        0  0.027561       0.50   0.000000
7          0  10032  1.000000    0  ...        0  0.187052       0.75   0.000000

[8 rows x 12 columns]


Запускаем

In [ ]:
if __name__ == '__main__':
    training_data = numpy_train
    model = LambdaMART(training_data, 20, 0.01)
    model.fit()

Tree 0
Epoch:0, Average NDCG : 0.9979443274266028
Tree 1
Epoch:1, Average NDCG : 0.9979443274266028
Tree 2
Epoch:2, Average NDCG : 0.9979443274266028
Tree 3
Epoch:3, Average NDCG : 0.9979443274266028
Tree 4
Epoch:4, Average NDCG : 0.9979443274266028
Tree 5
Epoch:5, Average NDCG : 0.9979443274266028
Tree 6
Epoch:6, Average NDCG : 0.9979443274266028
Tree 7
Epoch:7, Average NDCG : 0.9979443274266028
Tree 8
Epoch:8, Average NDCG : 0.9979443274266028
Tree 9
Epoch:9, Average NDCG : 0.9979443274266028
Tree 10
Epoch:10, Average NDCG : 0.9979443274266028
Tree 11
Epoch:11, Average NDCG : 0.9979443274266028
Tree 12
Epoch:12, Average NDCG : 0.9979443274266028
Tree 13
Epoch:13, Average NDCG : 0.9979443274266028
Tree 14
Epoch:14, Average NDCG : 0.9979443274266028
Tree 15
Epoch:15, Average NDCG : 0.9979443274266028
Tree 16
Epoch:16, Average NDCG : 0.9979443274266028
Tree 17
Epoch:17, Average NDCG : 0.9979443274266028
Tree 18
Epoch:18, Average NDCG : 0.9979443274266028
Tree 19
Epoch:19, Average NDCG :

In [ ]:
#сравним значения на тренировочной и тестовой выборке   
#тренировочная
k = 10
test_data = numpy_train
ndcg = model.validate(test_data, k)


Epoch:10, Average NDCG : 0.9977069382261459


In [ ]:
#тестовая 
k= 10
test_data = numpy_test
ndcg = model.validate(test_data, k)

Данная релевантность:  [0. 0. 0. 2. 0. 0. 1. 0.]
Порядок того как бы мы упорядочили документы :  [4 7 6 1 2 0 5 3]
Как расположилась релевантность в предсказанном порядке:  [0. 0. 1. 0. 0. 0. 0. 2.]
Epoch:10, Average NDCG : 0.5444679136514768


ВЫВОД: 

Модель переобучена, т.к. на тренировочной выборке оценка качества близится к 1 

Также, можно сказать, что модель плохо работает, т.к. качество на тестовой выборке варируется в пределах 0,4-0,5

Это можно увидеть на примере запроса 10032

Полученный вариант ранжирования не является идеальным т.к. релевантность расположена не от большего к меньшему, а в порядке ближе к хаотичному (самый релевантный документ стоит на последнем месте). 